# SageMaker Demo: Employee Attrition Prediction Using Feature Store and XGBoost

This notebook demonstrates how to use Amazon SageMaker's Feature Store and XGBoost built-in algorithm to predict employee attrition.

In [ ]:
# Step 1: Setup
import sagemaker

from sagemaker.session import get_execution_role

import boto3
import pandas as pd

sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = boto3.Session().region_name

# S3 bucket for storing data
bucket = 'sagemaker-ml3'

# Load the dataset
file_path = 'Employee.csv'  # Replace with your actual file path in S3 if needed
employee_df = pd.read_csv(file_path)
employee_df.head()

In [ ]:
# Step 2: Data Preparation
# Convert categorical columns to numeric
employee_df['Education'] = employee_df['Education'].astype('category').cat.codes
employee_df['City'] = employee_df['City'].astype('category').cat.codes
employee_df['Gender'] = employee_df['Gender'].astype('category').cat.codes
employee_df['EverBenched'] = employee_df['EverBenched'].map({'Yes': 1, 'No': 0})

# Drop rows with NaN values in the target column
employee_df.dropna(subset=['LeaveOrNot'])

# Convert target column to numeric if needed
employee_df['LeaveOrNot'] = employee_df['LeaveOrNot'].astype(int)

# Ensure no missing values in feature columns
employee_df = employee_df.dropna()

# Verify all columns are numeric
print(employee_df.dtypes)

# Define features and target
feature_columns = [
    'Education', 'JoiningYear', 'City', 'PaymentTier', 'Age',
    'Gender', 'EverBenched', 'ExperienceInCurrentDomain'
]
target_column = 'LeaveOrNot'

employee_df = employee_df[[target_column] + feature_columns]



# Display the transformed dataset
employee_df.head()

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from time import gmtime, strftime, sleep

# Create a Feature Group
feature_group_name = 'employee-feature-group-' + strftime('%Y%m%d%H%M%S', gmtime())
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

# Define the schema
record_identifier_name = 'EmployeeID'  # Unique identifier for records
event_time_feature_name = 'EventTime'  # Column representing the time of event

# Ensure EventTime is in the correct ISO-8601 format
employee_df[event_time_feature_name] = pd.to_datetime('now').strftime('%Y-%m-%dT%H:%M:%S.%fZ')
employee_df[record_identifier_name] = employee_df.index

# Load features to the Feature Store
feature_group.load_feature_definitions(data_frame=employee_df)

# Enable the Online Store when creating the Feature Group
feature_group.create(
    s3_uri=f's3://{bucket}/features',
    record_identifier_name=record_identifier_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True  # Enable the Online Store
)

In [ ]:
# Check the status of the Feature Group
status = feature_group.describe().get("FeatureGroupStatus")
print(f"Feature Group Status: {status}")

if status == "Created":
    print("Feature Group is Created and ready for use. Proceeding with ingestion...")
    
    # Ingest data into the Feature Store
    feature_group.ingest(data_frame=employee_df, max_workers=3, wait=True)
    print('Data ingested into Feature Store.')

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sklearn.model_selection import train_test_split 

# Initialize the SageMaker Feature Store runtime client
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')

# Define the feature group name and features you want to retrieve
feature_names = ['Education', 'JoiningYear', 'City', 'PaymentTier', 'Age', 'Gender', 'EverBenched', 'ExperienceInCurrentDomain', 'LeaveOrNot']

# Retrieve records and convert to DataFrame
records = []
for record_id in employee_df.index.astype(str):
    response = featurestore_runtime.get_record(
        FeatureGroupName=feature_group_name,
        RecordIdentifierValueAsString=str(record_id),
        FeatureNames=feature_names
    )
    # Check if 'Record' is in the response and add to records list
    if 'Record' in response:
        record = {feature['FeatureName']: feature['ValueAsString'] for feature in response['Record']}
        records.append(record)
    else:
        print(f"Record with ID {record_id} not found. Ensure previous step was completed")

# Convert the list of records to a DataFrame
retrieved_df = pd.DataFrame(records)

# Check if we have any retrieved records
if not retrieved_df.empty:
    # Split the data into training and test sets
    train_df, test_df = train_test_split(retrieved_df, test_size=0.2, random_state=42)
    print("Training and test data split after retrieval from Feature Store.")
else:
    print("No records retrieved. Please check the feature group and identifiers.")

## Train the Model Using Local Data with S3 Mode (Default)

In [ ]:
# Initialize S3 client
s3 = boto3.client('s3')

# Define your S3 bucket and prefix
prefix = 'input-data'

# Save the data locally first
train_file = 'train.csv'
validation_file = 'validation.csv'
train_df.to_csv(train_file, index=False)
test_df.to_csv(validation_file, index=False)

# Upload the data to S3
s3.upload_file(train_file, bucket, f'{prefix}/train/{train_file}')
s3.upload_file(validation_file, bucket, f'{prefix}/validation/{validation_file}')

print(f"Training data uploaded to s3://{bucket}/{prefix}/train/{train_file}")
print(f"Validation data uploaded to s3://{bucket}/{prefix}/validation/{validation_file}")


In [ ]:
import sagemaker
import boto3
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput

# Initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"reg:squarederror",
        "num_round":"50"}

# Set an output path where the trained model will be saved
prefix = 'demo-built-in-algorithm'
output_path = f's3://{bucket}/{prefix}/output'

# Retrieve the XGBoost image URI
region = boto3.Session().region_name  # Automatically get the region
xgboost_container = image_uris.retrieve("xgboost", region, "1.7-1")

# Construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=xgboost_container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.xlarge', 
                                          volume_size=5,  # 5 GB 
                                          output_path=output_path)

# Define the data type and paths to the training and validation datasets
content_type = "csv"
train_input = TrainingInput(f"s3://{bucket}/input-data/train/", content_type=content_type)
validation_input = TrainingInput(f"s3://{bucket}/input-data/validation/", content_type=content_type)

# Execute the XGBoost training job
estimator.fit({'train': train_input, 'validation': validation_input})